In [1]:
using Plots
# The GR plotting background is significantly faster
gr();

In [2]:
function v_target(t)
    return [1, sin(t*2*pi/10)]
    
    # This codes for a zigzag with period 10
    if t % 10 >= 10/2
        return [1, 1]
    else
        return [1, -1]
    end
end

function a_predator(t, k_r, p_r, k_m)
    # Acceleration is directed directly at the prey, and is scaled by the mass
    return normalize(p_r - k_r)/k_m
end

function mass_to_period(kozai_masses)
    # Euler's Method time resolution
    dt = 0.5
    time_max=1000000

    # Preallocate giant arrays and use them repeatedly for speed
    print("Allocating memory...")
    flush(STDOUT)
    kozai = zeros(Int(time_max/dt+1),2)
    y_s = zeros(time_max)
    results = []
    print(" done.\n")
    flush(STDOUT)
    
    for kozai_mass in kozai_masses
        # Set initial conditions
        p_r = [0, 0]
        k_r = [0, 0]
        k_v = [0, 0]
        
        # This is just a fairly general Euler's Method solver for parametric DEs
        for (index, t) in enumerate(0:dt:time_max)
            p_r += v_target(t) .* dt
            k_v += a_predator(t, k_r, p_r, kozai_mass) .* dt

            if norm(k_v) != 0
                k_v = normalize(k_v) .* min(norm(k_v))
            end

            k_r += k_v .* dt

            kozai[index,:] = k_r
        end
        
        # Now we "flatten" the parametric position array [(x, y)...]
        # into a x vs. y array [y1, y2 ...]
        # NOTE: This only makes sense when the parametric curve passes
        # the vertical line test, which, in many cases, it does not.
        fill(y_s, 0)
        for i in 1:size(kozai,1)
            (x, y) = kozai[i,:]
            if ceil(Int, x) > time_max
                # If, as happens by chance, the prey passes max_time,
                # truncate to fit.
                continue
            end
            y_s[ceil(Int, x)] = round(Int,y)
        end
        
        # Make sure the trajectory is centered around zero so the signal is clean
        y_s -= mean(y_s)
        
        # Now convert to frequency space using a Fast Fourier Transform
        # Lots of random constants floating around to make the frequency mappings work.
        dT = 1 # This is 1 because we are working in index space
        yf = 2.0/(time_max/dT) * abs(rfft(y_s))
        xf = linspace(0, 0.5/dT, div(time_max/dT,2))
        
        # A heuristic to extract the "low" frequency modulation. Essentially, 
        # we find the strongest signal, which is the  "high" frequency 
        # oscillation and look for a peak at less than half its frequency.
        period = 1/xf[indmax(yf[10:div(indmax(yf),2)])+9]
        push!(results, period)
        
        @printf("%d\r", kozai_mass)
        flush(STDOUT)
    end
    results
end

function polyfit(x, y, n)
    # Really simple polynomial least-squares fit
    A = [ float(x[i])^p for i = 1:length(x), p = 0:n ]
    A \ y
end

function model(coeff)
    # Produce a model closure from the polyfit output
    function result(x)
        reduce(+, (c*x^(p-1) for (p, c) in enumerate(coeff)))
    end
end;

In [14]:
p_r = [0, 0]
k_r = [0, 0]
k_v = [0, 0]

dt = 0.5

kozai_mass=140
time_max=1000000
kozai = zeros(Int(time_max/dt+1),2)

prey = []

for (index, t) in enumerate(0:dt:time_max)
    p_r += v_target(t) .* dt
    k_v += a_predator(t, k_r, p_r, kozai_mass) .* dt

    if norm(k_v) != 0
        k_v = normalize(k_v) .* min(norm(k_v))
    end

    k_r += k_v .* dt

    kozai[index,:] = k_r
    push!(prey, p_r)
end
@printf("%d frames\n", size(kozai,1))

2000001 frames


In [18]:
plot(kozai[1:min(5000,end),1], kozai[1:min(5000,end),2])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 -25 
 
 
 0 
 
 
 25 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 30.6037,214.074 30.6045,214.073 30.6057,214.069 30.6073,214.063 30.6092,214.052 30.6115,214.038 30.6141,214.019 30.6171,213.996 30.6204,213.969 30.624,213.937 
 30.628,213.902 30.6323,213.862 30.6371,213.82 30.6422,213.776 30.6477,213.729 30.6536,213.682 30.66,213.633 30.6667,213.585 30.6738,213.536 30.6814,213.487 
 30.6893,213.439 30.6977,213.39 30.7065,213.341 30.7157,213.292 30.7252,213.242 30.7352,213.191 30.7456,213.139 30.7564,213.085 30.7676,213.03 30.7792,212.973 
 30.7912,212.915 30.8036,212.855 30.8164,212.794 30.8296,212.732 30.8432,212.67 30.8572,212.607 30.8716,212.543 30.8864,212.48 30.9016,212.417 30.9173,212.354 
 30.9333,212.291 30.9498,212.228 30.9666,212.165 30.9839,212.102 31.0016,212.038 31.0197,211.975 31.0382,211.91 31.0571,211.845 31.0764,211.779 31.0961,211.712 
 31.1162,211.644 31.1367,211.575 31.1576,211.505 31.1789,211.435 31.2007,211.365 31.2228,211.294 31.2454,211.224 31.2683,211.153 31.2917,211.083 31.3155,211.013 
 31.3396,210.943 31.3642,210.873 31.3892,210.803 31.4146,210.733 31.4404,210.663 31.4667,210.593 31.4933,210.522 31.5203,210.451 31.5478,210.379 31.5756,210.307 
 31.6039,210.234 31.6325,210.161 31.6616,210.087 31.691,210.012 31.7209,209.938 31.7512,209.864 31.7819,209.789 31.813,209.715 31.8445,209.641 31.8764,209.567 
 31.9088,209.493 31.9415,209.419 31.9746,209.346 32.0082,209.273 32.0421,209.199 32.0765,209.126 32.1113,209.052 32.1464,208.978 32.182,208.903 32.218,208.828 
 32.2544,208.753 32.2912,208.677 32.3284,208.602 32.366,208.525 32.4041,208.449 32.4425,208.373 32.4813,208.297 32.5206,208.221 32.5602,208.145 32.6003,208.069 
 32.6408,207.994 32.6817,207.919 32.7229,207.844 32.7646,207.769 32.8067,207.694 32.8492,207.62 32.8922,207.545 32.9355,207.469 32.9792,207.394 33.0234,207.318 
 33.0679,207.242 33.1129,207.166 33.1582,207.089 33.204,207.013 33.2502,206.936 33.2967,206.859 33.3437,206.783 33.3911,206.707 33.4389,206.631 33.4871,206.555 
 33.5358,206.479 33.5848,206.404 33.6342,206.329 33.6841,206.254 33.7343,206.18 33.785,206.105 33.836,206.03 33.8875,205.955 33.9394,205.88 33.9917,205.804 
 34.0444,205.729 34.0975,205.653 34.151,205.577 34.2049,205.501 34.2592,205.425 34.314,205.35 34.3691,205.274 34.4246,205.199 34.4806,205.124 34.537,205.049 
 34.5937,204.974 34.6509,204.9 34.7085,204.826 34.7665,204.752 34.8249,204.678 34.8837,204.605 34.9429,204.531 35.0025,204.457 35.0625,204.383 35.123,204.309 
 35.1838,204.235 35.2451,204.161 35.3067,204.087 35.3688,204.012 35.4313,203.938 35.4942,203.864 35.5574,203.79 35.6211,203.717 35.6852,203.643 35.7498,203.57 
 35.8147,203.497 35.88,203.425 35.9457,203.353 36.0119,203.281 36.0784,203.209 36.1454,203.138 36.2127,203.066 36.2805,202.994 36.3487,202.922 36.4173,202.851 
 36.4863,202.779 36.5557,202.707 36.6255,202.635 36.6957,202.563 36.7663,202.491 36.8373,202.419 36.9088,202.348 36.9806,202.277 37.0529,202.206 37.1255,202.135 
 37.1986,202.065 37.2721,201.995 37.346,201.926 37.4202,201.856 37.4949,201.787 37.57,201.718 37.6456,201.649 37.7215,201.581 37.7978,201.512 37.8745,201.443 
 37.9517,201.374 38.0292,201.305 38.1072,201.236 38.1855,201.167 38.2643,201.098 38.3435,201.029 38.4231,200.961 38.5031,200.893 38.5835,200.825 38.6643,200.758 
 38.7455,200.691 38.8271,200.624 38.9091,200.558 38.9916,200.492 39.0744,200.426 39.1577,200.361 39.2413,200.295 39.3254,200.229 39.4099,200.164 39.4948,200.098 
 39.58,200.033 39.6657,199.968 39.7518,199.902 39.8384,199.837 39.9253,199.772 40.0126,199.707 40.1003,199.642 40.1885,199.578 40.277,199.514 40.366,199.45 
 40.4553,199.387 40.5451,199.324 40.6353,199.262 40.7259,199.2 40.8169,199.138 40.9083,199.076 41.0001,199.014 41.0923,198.953 41.1849,198.891 41.2779,198.83 
 41.3714,198.768 41.4652,19

In [16]:
#x_max = Int(ceil(maximum(kozai[:,1])))
y_s = zeros(time_max)
for i in 1:size(kozai,1)
    (x, y) = kozai[i,:]
    if ceil(Int, x) > time_max
        continue
    end
    y_s[ceil(Int, x)] = round(Int,y)
end
y_s -= mean(y_s)
dT = 1
yf = 2.0/(time_max/dT) * abs(rfft(y_s))
xf = linspace(0, 0.5/dT, div(time_max/dT,2))

pks= zeros(length(xf))
pks[findmax(yf[10:div(findmax(yf)[2],2)])[2]+9] = 0.1
1/xf[findmax(yf[10:div(findmax(yf)[2],2)])[2]+9]

9615.365384615385

In [17]:
plot_crop = 1000
plot(xf[1:min(plot_crop,end)], yf[1:min(plot_crop,end)])
plot!(xf[1:min(plot_crop,end)], pks[1:min(plot_crop,end)])

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00000 
 
 
 0.00025 
 
 
 0.00050 
 
 
 0.00075 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:0.8; stroke-opacity:1; fill:none" points="
 39.4926,384.952 40.0497,349.591 40.6068,376.232 41.1639,370.349 41.7211,377.883 42.2782,372.021 42.8353,373.261 43.3925,376.399 43.9496,373.258 44.5067,367.137 
 45.0638,380.517 45.621,370.714 46.1781,376.656 46.7352,377.406 47.2923,381.181 47.8495,373.57 48.4066,379.61 48.9637,376.433 49.5209,377.372 50.078,362.468 
 50.6351,370.311 51.1922,363.589 51.7494,380.9 52.3065,364.243 52.8636,377.215 53.4208,372.055 53.9779,370.872 54.535,370.452 55.0921,379.981 55.6493,382.159 
 56.2064,370.775 56.7635,370.821 57.3206,356.055 57.8778,361.662 58.4349,379.185 58.992,375.053 59.5492,369.041 60.1063,375.519 60.6634,358.034 61.2205,373.889 
 61.7777,370.502 62.3348,380.902 62.8919,375.664 63.449,375.938 64.0062,368.002 64.5633,370.946 65.1204,363.18 65.6776,369.74 66.2347,366.293 66.7918,378.926 
 67.3489,376.784 67.9061,370.406 68.4632,370.242 69.0203,370.688 69.5775,365.093 70.1346,376.639 70.6917,376.906 71.2488,361.482 71.806,373.376 72.3631,358.858 
 72.9202,367.553 73.4773,376.738 74.0345,366.211 74.5916,370.028 75.1487,380.168 75.7059,374.188 76.263,383.177 76.8201,376.681 77.3772,373.841 77.9344,359.33 
 78.4915,378.973 79.0486,366.728 79.6057,370.881 80.1629,365.162 80.72,356.611 81.2771,355.051 81.8343,368.176 82.3914,367.132 82.9485,367.706 83.5056,372.008 
 84.0628,356.475 84.6199,361.527 85.177,351.896 85.7342,379.563 86.2913,366.324 86.8484,369.67 87.4055,370.378 87.9627,356.307 88.5198,367.751 89.0769,367.013 
 89.634,356.17 90.1912,375.02 90.7483,363.622 91.3054,375.249 91.8626,360.768 92.4197,340.504 92.9768,354.939 93.5339,351.398 94.0911,352.471 94.6482,287.998 
 95.2053,194.109 95.7624,184.816 96.3196,89.1529 96.8767,351.459 97.4338,3.93701 97.991,83.7806 98.5481,354.574 99.1052,288.835 99.6623,305.653 100.219,350.534 
 100.777,328.632 101.334,358.557 101.891,349.269 102.448,351.99 103.005,352.747 103.562,378.202 104.119,343.862 104.676,364.687 105.234,355.876 105.791,376.95 
 106.348,360.898 106.905,366.519 107.462,348.726 108.019,367.739 108.576,349.133 109.134,370.089 109.691,358.86 110.248,361.002 110.805,338.967 111.362,369.252 
 111.919,342.652 112.476,375.216 113.033,374.09 113.591,352.869 114.148,369.405 114.705,337.652 115.262,358.316 115.819,358.402 116.376,367.634 116.933,354.726 
 117.49,371.586 118.048,360.761 118.605,360.123 119.162,377.406 119.719,352.618 120.276,346.97 120.833,368.798 121.39,344.017 121.947,340.539 122.505,341.481 
 123.062,347.65 123.619,371.875 124.176,351.913 124.733,345.425 125.29,360.698 125.847,344.724 126.404,352.427 126.962,359.417 127.519,359.237 128.076,358.02 
 128.633,367.182 129.19,353.805 129.747,370.083 130.304,369.588 130.861,367.074 131.419,361.634 131.976,357.468 132.533,348.731 133.09,371.574 133.647,378.812 
 134.204,380.079 134.761,347.317 135.319,342.172 135.876,362.665 136.433,353.635 136.99,352.671 137.547,380.792 138.104,355.803 138.661,340.592 139.218,376.619 
 139.776,359.468 140.333,376.867 140.89,355.633 141.447,325.988 142.004,334.5 142.561,341.649 143.118,350.95 143.675,358.43 144.233,365.068 144.79,311.39 
 145.347,348.676 145.904,346.133 146.461,327.995 147.018,365.327 147.575,348.752 148.132,372.537 148.69,360.493 149.247,371.706 149.804,363.016 150.361,327.987 
 150.918,333.668 151.475,341.76 152.032,306.09 152.589,338.098 153.147,338.697 153.704,326.83 154.261,340.196 154.818,303.981 155.375,326.759 155.932,278.105 
 156.489,365.194 157.046,289.45 157.604,302.143 158.161,328.085 158.718,352.301 159.275,334.627 159.832,329.379 160.389,356.951 160.946,366.249 161.503,341.12 
 162.061,351.182 162.618,351.02 163.175,326.101 163.732,366.598 164.289,370.979 164.846,380.215 165.403,356.174 165.961,310.649 166.518,329.096 167.075,346.477

In [28]:
anim_dt=10
@gif for i in 1:anim_dt:4000
    @printf("%d\r", i)
    flush(STDOUT)
    plot(map(x -> x[1], prey[1:i]), map(x -> x[2], prey[1:i]))
    plot!(kozai[1:i,1], kozai[1:i,2])
end every 1

INFO: Saved animation to /home/pythonnut/Documents/prog/sipz/tmp.gif


Plots.AnimatedGif("/home/pythonnut/Documents/prog/sipz/tmp.gif")

In [ ]:
X = 40:1:300
periods = mass_to_period(X);

#for x in zip(X, periods)
#    @printf("%d,%f\n", x[1], x[2])
#end

Allocating memory... done.


In [ ]:
plot(X, periods)
fit = polyfit(X, periods,2)
plot!(X, map(model(fit), X))

In [ ]:
plot(X, periods- map(model(fit), X))

In [ ]:
fit